In [2]:
import cv2
import numpy as np

image = cv2.imread("images/1.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
brightened = cv2.addWeighted(gray, 1.15, np.zeros(gray.shape, gray.dtype), 0, 25)
blurred = cv2.bilateralFilter(brightened, 9, 100, 100)
cv2.imshow("blurred", blurred)
cv2.waitKey(0)
cv2.destroyAllWindows()

edged = cv2.Canny(blurred, 20, 25, L2gradient=True)

# define a (3, 3) structuring element
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

# apply the dilation operation to the edged image
dilate = cv2.dilate(edged, kernel, iterations=1)
cv2.imshow("Dilated image", dilate)
cv2.waitKey(0)
cv2.destroyAllWindows()

# find the contours in the dilated image
contours, _ = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
image_copy = image.copy()
# draw the contours on a copy of the original image
cv2.drawContours(image_copy, contours, -1, (0, 255, 0), 2)
print(len(contours), "objects were found in this image.")

cv2.imshow("contours", image_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

117 objects were found in this image.


In [3]:
import cv2
import numpy as np

# Creating a VideoCapture object to read the video
cap = cv2.VideoCapture('videos/1.avi')
# get the video frame height and width
#frame_width = int(cap.get(3))
#frame_height = int(cap.get(4))
#save_name = f"videos/1_output.mp4"
# define codec and create VideoWriter object
#out = cv2.VideoWriter(
#    save_name,
#    cv2.VideoWriter_fourcc(*'mp4v'), 10, 
#    (frame_width, frame_height)
#)
# Flag to indicate if the video is paused
paused = False

# Loop until explicitly stopped
while True:
    # Pause or resume video playback
    key = cv2.waitKey(1)
    if key == ord(' '):  # Spacebar key
        paused = not paused

    # Exit the loop when 'q' key is pressed
    if key == ord('q'):
        break
        
    if paused:
        continue
    # Check if the video capture has reached the end
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        # Reset video capture to restart the loop
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    # Capture frame-by-frame
    ret, frame = cap.read()

    # Check if frame was successfully captured
    if not ret:
        break      

    img = frame
    height, width = img.shape[:2]
    y = np.ones((height, width), np.uint8) * 128
    output = np.zeros((height, width), np.uint8)
    # generating the kernels
    kernel1 = np.array([[0, -1, -1], # kernel for embossing bottom left side
                        [1, 0, -1],
                        [1, 1, 0]])
    kernel2 = np.array([[-1, -1, 0], # kernel for embossing bottom right side
                        [-1, 0, 1],
                        [0, 1, 1]])
    # you can generate kernels for embossing top as well
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    output1 = cv2.add(cv2.filter2D(gray, -1, kernel1), y) # emboss on bottom left side
    output2 = cv2.add(cv2.filter2D(gray, -1, kernel2), y) # emboss on bottom right side
    for i in range(height):
        for j in range(width):
            output[i, j] = max(output1[i, j], output2[i, j]) # combining both embosses to produce stronger emboss

    brightened = cv2.addWeighted(output, 1.15, np.zeros(gray.shape, gray.dtype), 0, 25)
    blurred = cv2.bilateralFilter(brightened, 9, 100, 100)


    edged = cv2.Canny(blurred, 20, 25, L2gradient=True)

    # define a (3, 3) structuring element
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    # apply the dilation operation to the edged image
    dilate = cv2.dilate(edged, kernel, iterations=1)

    # find the contours in the dilated image
    contours, _ = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    image_copy = img.copy()
    # draw the contours on a copy of the original image
    cv2.drawContours(image_copy, contours, -1, (0, 255, 0), 2)
    print(len(contours), "objects were found in this image.")

    #cv2.imshow("final", image_copy)
    #out.write(image_copy)
    #if cv2.waitKey(100) & 0xFF == ord('q'):
    #    break

    # save each frame as a .jpg image
    count = cap.get(cv2.CAP_PROP_POS_FRAMES)
    cv2.imwrite(f"images/{count}.jpg", image_copy)
    count += 1

# Release the video capture object
cap.release()

# Closes all the windows currently opened.
cv2.destroyAllWindows()


216 objects were found in this image.
217 objects were found in this image.
189 objects were found in this image.
216 objects were found in this image.
204 objects were found in this image.
192 objects were found in this image.
219 objects were found in this image.
193 objects were found in this image.
203 objects were found in this image.
196 objects were found in this image.
203 objects were found in this image.
218 objects were found in this image.


KeyboardInterrupt: 